In [ ]:
import numpy as np
import pyvista as pv
import shapely as sh
import matplotlib.pyplot as plt

from airfoil.util.array_helpers import split_indexable
from airfoil.util.shapely_helpers import (
    plot_shapely_directional,
    plot_shapely
)
from airfoil.util.linestring_helpers import (
    deflection_angle,
    resample_spline_fallback_linear,
    ensure_closed,
    remove_sequential_duplicates,
    split_and_roll_at_top_right,
    resample_shapes
)
from airfoil.util.pyvista_helpers import (
    create_ruled_surface,
    mesh_from_polygon
)
from airfoil.cnc.cnc_machine_mesh import draw_machine
from airfoil.cnc import (
    states_to_3d_points,
    state_to_3d_points
)

In [ ]:
# create components
s0 = sh.Point(0,0).buffer(60)
s1 = sh.Point(0,0).buffer(60+30)
s2 = sh.box(-100,-100,100,0)
s3 = sh.box(-15,50, 15,250)
clip_circle = sh.Point(0,100).buffer(90)

shapea_p = sh.affinity.scale(
    sh.difference(sh.union(s3,sh.difference(s1,s2)),s0),
    0.5,0.5, origin=(0,0)
)
shapeb_p = sh.affinity.rotate(sh.intersection(shapea_p,clip_circle),10)
    
shapea,shapeb = resample_shapes(
    [
        np.array(shapea_p.exterior.coords),
        np.array(shapeb_p.exterior.coords),
    ],
    target_length=5
)

shapea_p, shapeb_p = sh.Polygon(shapea),sh.Polygon(shapeb)
shapec_p, shaped_p = [
    sh.affinity.translate(sh.affinity.rotate(sh.Polygon(shapeb),170), 30,-120),
    sh.affinity.translate(sh.affinity.rotate(sh.Polygon(shapea),190), 50,-100),
]


plot_shapely_directional([
        shapea_p, 
        shapeb_p, 
        shapec_p, 
        shaped_p
    ],
    legend=["a","b","c","d"]
)

In [ ]:
def make_mesh_from_side_surfaces(a:np.ndarray, b:np.ndarray, foam_width:float=150):

    shapea3d = np.insert(a, 0, -foam_width/2, axis=-1)
    shapeb3d = np.insert(b, 0,  foam_width/2, axis=-1)

    aa = mesh_from_polygon(sh.Polygon(a)).translate((-foam_width/2,0,0))
    bb = mesh_from_polygon(sh.Polygon(b)).translate(( foam_width/2,0,0))
    aabb = create_ruled_surface(shapea3d, shapeb3d)

    result =  pv.merge([aa,bb,aabb])
    assert result.is_manifold
    return result

In [ ]:
meesh = make_mesh_from_side_surfaces(shapea, shapeb)
meesh2 = make_mesh_from_side_surfaces(
    np.array(shapec_p.exterior.coords),
    np.array(shaped_p.exterior.coords),
)

(_,_,yl,yu,zl,zu) = (meesh+meesh2).bounds

x = np.random.uniform(yl-5, yu+5, 1000)
y = np.random.uniform(zl-5, zu+5, 1000)
z = np.random.uniform(yl-5, yu+5, 1000)
a = np.random.uniform(zl-5, zu+5, 1000)

aa,bb = states_to_3d_points(x,y,z,a)

combo = (meesh+meesh2)
rez = []
for aai,bbi in zip(aa,bb):
    pnt,cell = combo.ray_trace(aai,bbi)
    rez.append(len(pnt)==0)
aa = aa[rez]
bb = bb[rez]

pt = pv.Plotter()
for aai,bbi in zip(aa,bb):
    pt.add_mesh(pv.Line(aai,bbi), color="pink", opacity=0.5)


target_state = np.array([yu+2,0,yu+2,0])

pt.add_mesh(pv.Line(*state_to_3d_points(*target_state)), color="blue")

pt.add_mesh(meesh)
pt.add_mesh(meesh2)
draw_machine(-10,0,-10,0,spacing=220, pt=pt, opacity=0.1)
pt.show()

In [ ]:
import numpy as np
from typing import Iterable, Tuple
import pyvista as pv

# Assuming your AStar library is imported
from astar import find_path

class StateSpacePathfinder:
    def __init__(self, mesh_combo, bounds, step_size=1.0):
        self.mesh_combo = mesh_combo
        self.bounds = bounds  # (xl, xu, yl, yu, zl, zu)
        self.step_size = step_size
        
    def is_valid_state(self, state: Tuple[float, float, float, float]) -> bool:
        """Check if state is within bounds and collision-free"""
        x, y, z, a = state
        xl, xu, yl, yu, zl, zu = self.bounds
        
        # Bounds check
        if not (xl <= x <= xu and yl <= y <= yu and zl <= z <= zu and zl <= a <= zu):
            return False
            
        # Collision check using your existing function
        try:
            aa, bb = states_to_3d_points([x], [y], [z], [a])
            pnt, cell = self.mesh_combo.ray_trace(aa[0], bb[0])
            return len(pnt) == 0  # No collision if no intersection points
        except:
            return False
    
    def get_neighbors(self, state: Tuple[float, float, float, float]) -> Iterable[Tuple[float, float, float, float]]:
        """Generate valid neighboring states"""
        x, y, z, a = state
        
        # 4D movement directions (26 neighbors in 4D, similar to 3D cube)
        directions = []
        for dx in [-self.step_size, 0, self.step_size]:
            for dy in [-self.step_size, 0, self.step_size]:
                for dz in [-self.step_size, 0, self.step_size]:
                    for da in [-self.step_size, 0, self.step_size]:
                        if dx == dy == dz == da == 0:
                            continue
                        directions.append((dx, dy, dz, da))
        
        neighbors = []
        for dx, dy, dz, da in directions:
            new_state = (x + dx, y + dy, z + dz, a + da)
            if self.is_valid_state(new_state):
                neighbors.append(new_state)
        
        return neighbors
    
    def heuristic(self, current: Tuple[float, float, float, float], 
                  goal: Tuple[float, float, float, float]) -> float:
        """Euclidean distance in 4D space"""
        return np.linalg.norm(np.array(current) - np.array(goal))
    
    def distance(self, state1: Tuple[float, float, float, float], 
                 state2: Tuple[float, float, float, float]) -> float:
        """Distance between adjacent states"""
        return np.linalg.norm(np.array(state1) - np.array(state2))

# Main pathfinding function
def find_4d_path(start_state, target_state, mesh_combo, bounds, step_size=1.0):
    """Find A* path in 4D state space"""
    
    pathfinder = StateSpacePathfinder(mesh_combo, bounds, step_size)
    
    # Convert numpy arrays to tuples for hashing
    start = tuple(start_state)
    goal = tuple(target_state)
    
    # Use the provided find_path function
    path = find_path(
        start=start,
        goal=goal,
        neighbors_fnct=pathfinder.get_neighbors,
        heuristic_cost_estimate_fnct=pathfinder.heuristic,
        distance_between_fnct=pathfinder.distance,
        is_goal_reached_fnct=lambda current, target: np.linalg.norm(np.array(current) - np.array(target)) < step_size
    )
    
    return list(path) if path else None

# Usage with your existing code:
# Get bounds from your mesh
combo = (meesh + meesh2)
xl, xu, yl, yu, zl, zu = combo.bounds

# Extend bounds slightly for search space
bounds = (xl-5, xu+5, yl-5, yu+5, zl-5, zu+5)

# Find path
start_state = np.array([0, 0, 0, 0])
path = find_4d_path(start_state, target_state, combo, bounds, step_size=2.0)

if path:
    print(f"Path found with {len(path)} waypoints")
    
    # Visualize the path
    pt = pv.Plotter()
    
    # Add existing meshes
    pt.add_mesh(meesh)
    pt.add_mesh(meesh2)
    
    # Add path visualization
    for i, state in enumerate(path):
        aa_path, bb_path = states_to_3d_points([state[0]], [state[1]], [state[2]], [state[3]])
        color = "red" if i == 0 else "green" if i == len(path)-1 else "yellow"
        pt.add_mesh(pv.Line(aa_path[0], bb_path[0]), color=color, line_width=3)
    
    # Add target
    pt.add_mesh(pv.Line(*state_to_3d_points(*target_state)), color="blue")
    
    # Add machine visualization
    draw_machine(-10, 0, -10, 0, spacing=220, pt=pt, opacity=0.1)
    
    pt.show()
else:
    print("No path found")